Requirement:

>      4. The notebook formulas that can import the CSV contents from step 2, and analyze the new headlines/articles, then give them a truthfulness rating. For example: "Likely true", "likely false", etc.




In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras import models
import texthero as hero
import pandas as pd
import sqlite3

Load Model

In [9]:
fp = './tf_saved_model/fake_news_model.h5'
bloomberg_model = tf.keras.models.load_model(fp)

Connect to SQLite3

In [32]:
con = sqlite3.connect('bloomberg_news.db')
data = pd.read_sql("SELECT * FROM news", con)
data.head()

,author,content,description,publishedAt,source,title,url,urlToImage
0,Brandon Kochkodin,None,Now’s a great time to get into Bitcoin mining....,2021-07-07,Bloomberg,Bitcoin Miners Thwarted by Data Center Crunch ...,https://www.bloomberg.com/news/articles/2021-0...,https://assets.bwbx.io/images/users/iqjWHBFdfx...
1,Charlie Wells,None,The rules of investing are getting a shakeup. ...,2021-07-15,Bloomberg,Bloomberg Wealth: Your Investing Tactics May B...,https://www.bloomberg.com/news/newsletters/202...,https://assets.bwbx.io/images/users/iqjWHBFdfx...
2,"Georgina McKay, Tracy Alloway",None,India’s Covid death toll has been massively un...,2021-07-21,Bloomberg,Five Things You Need to Know to Start Your Day,https://www.bloomberg.com/news/newsletters/202...,https://assets.bwbx.io/images/users/iqjWHBFdfx...
3,Spencer Soper,None,Speculation that the company would allow crypt...,2021-07-30,Bloomberg,Amazon Has 1.55 Trillion Reasons to Start Acce...,https://www.bloomberg.com/news/newsletters/202...,https://assets.bwbx.io/images/users/iqjWHBFdfx...
4,Joanna Ossinger,None,"Superficially, it should be a perfect match. I...",2021-07-10,Bloomberg,Crypto Demand From Millennials Poses Big Test ...,https://www.bloomberg.com/news/articles/2021-0...,https://assets.bwbx.io/images/users/iqjWHBFdfx...


Preprocess Data

In [33]:
data = data[['title', 'description']]

Drop NaNs

In [34]:
if data.isna().sum().sum() >= 1:
    data.dropna(inplace=True)

In [35]:
data['description'] = hero.clean(data.title + ' ' +data.description)
data = data.drop('title', axis=1)

Tokenizer

In [42]:
def tokenize_description(x: List[str]) -> List[int] and Tokenizer:
    """
    :param x:
    :returns:
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    X = tokenizer.texts_to_sequences(x)
    tk_model = tokenizer
    
    return X, tk_model

In [43]:
def padding_sequence(x, maxlen=1000):
    X = pad_sequences(x, maxlen=maxlen)
    return X

In [47]:
X = data['description']
X, tk_model = tokenize_description(X)
X = padding_sequence(X)

## Predictions

---

In [66]:
preds = (bloomberg_model.predict(X) >= 0.5).astype(int)
preds = pd.Series(np.ravel(preds))
pred_df = pd.concat([data.description, preds], axis=1)
pred_df = pred_df.rename(columns={0: 'fake_0'})

In [70]:
pred_df

,description,fake_0
0,bitcoin miners thwarted data center crunch ami...,1
1,bloomberg wealth investing tactics may date ru...,0
2,five things need know start day india' covid d...,0
3,amazon trillion reasons start accepting crypto...,0
4,crypto demand millennials poses big test robo ...,1
5,jeff bezos star wars inspire hunt trillion spa...,1
6,money stuff private equity firm got rich makin...,1
7,money stuff easy borrow money money could dumb...,1
8,five things need know start day china unveils ...,0
9,esports crypto streaming service turns bandwid...,1


In [73]:
pred_df.style.background_gradient(
    subset='news_is', cmap='Reds_r')

,description,fake_0
0,bitcoin miners thwarted data center crunch amid profit boom ' great time get bitcoin mining find place plug,1
1,bloomberg wealth investing tactics may date rules investing getting shakeup sure know stonks bitcoin robinhood ' keenly aware called retail revolution,0
2,five things need know start day india' covid death toll massively underreported top lenders losing faith china evergrande musk loves bitcoin ' afraid show asia ' need know,0
3,amazon trillion reasons start accepting crypto speculation company would allow cryptocurrency payments roiled markets could really happen,0
4,crypto demand millennials poses big test robo advisers superficially perfect match interest cryptocurrencies surged among young tech literate exactly demographic targeted big robo advisers,1
5,jeff bezos star wars inspire hunt trillion space unicorn billionaires sci fi classics cathie wood spurring traders interstellar interests toward stocks etfs risks remain world,1
6,money stuff private equity firm got rich making private equity firms rich sneakily important fact finance make million year make million year company makes million year ' worth least say million stream income worth multiple year' inc...,1
7,money stuff easy borrow money money could dumb oversimplified model goes like,1
8,"five things need know start day china unveils sweeping overhaul billion education tech sector fauci warns u heading ""wrong direction"" delta tears across world china' economic recovery showing signs weakness ' need know",0
9,esports crypto streaming service turns bandwidth profits one biggest knocks cryptocurrencies real world benefits besides speculation,1
